In [3]:
import os
import glob
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
environ = os.environ
SOURCE2 = Path(environ.get('SOURCE2'))

xml_files = sorted(glob.glob(str(SOURCE2 / 'xml' / '*.xml')))
word_image_files = sorted(glob.glob(str(SOURCE2 / 'words' / '**' / '*.png'), recursive=True))

In [8]:
import tqdm
import multiprocessing as mp
import xml.etree.ElementTree as ET

from PIL import Image
from dataclasses import dataclass

@dataclass
class Source2Word:
    transcription: str
    id: str
    writer_id: str
    img_path: Path

def words_from_xml(xml_file):
    parsed = ET.parse(xml_file)
    root = parsed.getroot()

    root_id = root.get('id')
    writer_id = root.get('writer-id')
    xml_words = []

    for line in root.findall('handwritten-part')[0].findall('line'):
        for word in line.findall('word'):
            word_id = word.get('id')
            transcription = word.get('text')

            img_path = Path([f for f in word_image_files if f.endswith(word_id + '.png')][0])

            try:
                with Path.open(img_path) as _:
                    xml_words.append( 
                        Source2Word(
                            transcription=transcription,
                            writer_id=writer_id,
                            id=word_id,
                            img_path=img_path,
                        )
                    )

            except Exception:
                pass
    
    return xml_words

with mp.Pool(processes=mp.cpu_count()) as pool:
    words_from_xmls = list(
        tqdm.tqdm(
            pool.imap(words_from_xml, xml_files),
            total=len(xml_files),
            desc='Preparing dataset'
        )
    )

words = [word for words in words_from_xmls for word in words]
# words = []
# for words_list in words_from_xmls:
#     for word in words_list:
#         words.append(word)


Preparing dataset:   0%|          | 4/1539 [00:05<38:21,  1.50s/it]  


KeyboardInterrupt: 